This script is for cell cell interaction metrics generation

In [ ]:
import scanpy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
import gzip
import seaborn as sns 
import sklearn

In [ ]:
import squidpy as sq

In [ ]:
import time

In [ ]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi= 80)

# load annotated h5ad file

In [ ]:
file_path='/oak/stanford/groups/cgawad/home/Cancer_Studies/SC_RNA_SEQ/ALSF_AML/scanpy/H5AD/'

In [ ]:
Combo=sc.read_h5ad(file_path+ "ALSF_AML_Combo_3500_with_raw_count.h5ad")

# Receptor -ligand analysis

In [ ]:
file_path='/oak/stanford/groups/cgawad/home/Cancer_Studies/SC_RNA_SEQ/ALSF_AML/scanpy/Cell_interaction/'

In [ ]:
#remove noise from normal BM samples
adata=Combo[np.logical_not(Combo.obs["lineage_Sample"].isin(['AML',
                                                             #'AML-B','AML-Ery','AML-Mono','AML-T','AML-NK'
                                                            ]))]

In [ ]:
HBM_sample=['0_HealthyBM1','0_HealthyBM2']

In [ ]:
AML_sample=['AML335','AML647', 'AML882', 'AML948','AML1355',
            'AML3082', 'AML3121', 'AML3210',  'AML3371', 'AML3492', 
            'AML4000', 'AML4010', 'AML4035','AML4062', 'AML4068','AML4090',
            'AML4102', 'AML4116', 'AML4127', 'AML4192', 'AML4226', 
            'AML4232', 'AML4239', 'AML4264', 'AML4271', 'AML4304', 'AML4363'
            ]

In [ ]:
for AML in AML_sample:
    print(AML)
    AML_Sq =adata[adata.obs['SampleID'].isin([AML]),:]
    res = sq.gr.ligrec(
    AML_Sq,
    n_perms=2000,
    cluster_key="PAC_anno",
    complex_policy='min',
    copy=True,
    use_raw=False,
    threshold=0.005,
    n_jobs=1,
    interactions_params={'resources': ['CellPhoneDB','CellChatDB','CellCellInteractions']}
    )
    
    df=pd.DataFrame(res["pvalues"])
    
    res["pvalues"].to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_recepter_ligand_interaction_pval.csv'%AML)
    
    mean_df=pd.DataFrame(res["means"])
    
    res["means"].to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_recepter_ligand_interaction_mean.csv'%AML)
    
    pval_df=df
    pval_df.columns = ['|'.join(col) for col in pval_df.columns.values]
    pval_df = pval_df.reset_index() 
    melt_pval= pd.melt(pval_df, id_vars=['source','target'],value_name='pval')
    melt_pval=melt_pval.loc[melt_pval.pval.notnull()]
    melt_pval[['Sender','Receptor']] = melt_pval['variable'].str.split('|',expand=True)
    melt_pval['Sample']=AML
    melt_pval['Sender_source'] = melt_pval['Sender'] +"|"+ melt_pval['source']
    melt_pval['Receptor_target'] = melt_pval['Receptor'] + "|"+ melt_pval['target']
    melt_df=melt_pval[['Sample','Sender_source','Receptor_target','pval']]
    melt_df.to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_PAC_chord_figure_input_before_filter_pval.csv'%AML)
    
    mean_df.columns = ['|'.join(col) for col in mean_df.columns.values]
    mean_df = mean_df.reset_index() 
    melt_mean= pd.melt(mean_df, id_vars=['source','target'],value_name='mean')
    melt_mean["mean"] = pd.to_numeric(melt_mean['mean'])
    melt_mean=melt_mean[melt_mean['mean']>0]
    melt_mean[['Sender','Receptor']] = melt_mean['variable'].str.split('|',expand=True)
    melt_mean['Sample']=AML
    melt_mean['Sender_source'] = melt_mean['Sender'] +"|"+ melt_mean['source']
    melt_mean['Receptor_target'] = melt_mean['Receptor'] + "|"+ melt_mean['target']
    melt_mean=melt_mean[['Sample','Sender_source','Receptor_target','mean']]
    melt_mean.to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_PAC_chord_figure_input_before_filter_mean.csv'%AML)
    

In [ ]:
mean_files = [file for file in all_files if file.startswith('AML') & \
              file.endswith('filter_mean_lastestDataBase.csv')]

In [ ]:
mean_files

In [ ]:
pval_files = [file for file in all_files if file.startswith('AML') & \
              file.endswith('filter_pval_lastestDataBase.csv')]

In [ ]:
pval_files

In [ ]:
li = []

for filename in mean_files:
    df = pd.read_csv(file_path+filename,index_col=0)
    li.append(df)


In [ ]:
df

In [ ]:
frame = pd.concat(li, axis=0,ignore_index=True)
frame

In [ ]:
frame.to_csv(file_path+'AML_PAC_chord_figure_input_before_filter_mean.csv') 

In [ ]:
pval = []

for filename in pval_files:
    df = pd.read_csv(file_path+filename,index_col=0)
    pval.append(df)

In [ ]:
pval_frame = pd.concat(pval, axis=0, ignore_index=True)
pval_frame

In [ ]:
pval_frame.to_csv(file_path+'AML_PAC_chord_figure_input_before_filter_pval_lastestDataBase.csv') 

In [ ]:
for HBM in HBM_sample:
    print(HBM)
    HBM_Sq =adata[adata.obs['SampleID'].isin([HBM]),:]
    res = sq.gr.ligrec(
    HBM_Sq,
    n_perms=2000,
    cluster_key="PAC_anno",
    complex_policy='min',
    copy=True,
    use_raw=False,
    threshold=0.005,
    n_jobs=1,
    interactions_params={'resources': ['CellPhoneDB','CellChatDB','CellCellInteractions']}
    )
    
    df=pd.DataFrame(res["pvalues"])
    
    res["pvalues"].to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_recepter_ligand_interaction_pval.csv'%HBM)
    
    mean_df=pd.DataFrame(res["means"])
    
    res["means"].to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_recepter_ligand_interaction_mean.csv'%HBM)
    
    pval_df=df
    pval_df.columns = ['|'.join(col) for col in pval_df.columns.values]
    pval_df = pval_df.reset_index() 
    melt_pval= pd.melt(pval_df, id_vars=['source','target'],value_name='pval')
    melt_pval=melt_pval.loc[melt_pval.pval.notnull()]
    melt_pval[['Sender','Receptor']] = melt_pval['variable'].str.split('|',expand=True)
    melt_pval['Sample']=AML
    melt_pval['Sender_source'] = melt_pval['Sender'] +"|"+ melt_pval['source']
    melt_pval['Receptor_target'] = melt_pval['Receptor'] + "|"+ melt_pval['target']
    melt_df=melt_pval[['Sample','Sender_source','Receptor_target','pval']]
    melt_df.to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_PAC_chord_figure_input_before_filter_pval.csv'%HBM)
    
    mean_df.columns = ['|'.join(col) for col in mean_df.columns.values]
    mean_df = mean_df.reset_index() 
    melt_mean= pd.melt(mean_df, id_vars=['source','target'],value_name='mean')
    melt_mean["mean"] = pd.to_numeric(melt_mean['mean'])
    melt_mean=melt_mean[melt_mean['mean']>0]
    melt_mean[['Sender','Receptor']] = melt_mean['variable'].str.split('|',expand=True)
    melt_mean['Sample']=AML
    melt_mean['Sender_source'] = melt_mean['Sender'] +"|"+ melt_mean['source']
    melt_mean['Receptor_target'] = melt_mean['Receptor'] + "|"+ melt_mean['target']
    melt_mean=melt_mean[['Sample','Sender_source','Receptor_target','mean']]
    melt_mean.to_csv(file_path+'/Jun_8_2023_CC-interaction/%s_PAC_chord_figure_input_before_filter_mean.csv'%HBM)
    

In [ ]:
mean_files = [file for file in all_files if file.startswith('0_') & \
              file.endswith('filter_mean_lastestDataBase.csv')]

In [ ]:
pval_files = [file for file in all_files if file.startswith('0_') & \
              file.endswith('filter_pval_lastestDataBase.csv')]

In [ ]:
li = []

for filename in mean_files:
    df = pd.read_csv(file_path+filename,index_col=0)
    li.append(df)


In [ ]:
frame = pd.concat(li, axis=0,ignore_index=True)
frame

In [ ]:
frame.to_csv(file_path+'Healthy_PAC_chord_figure_input_before_filter_mean_lastestDataBase.csv') 

In [ ]:
pval = []

for filename in pval_files:
    df = pd.read_csv(file_path+filename,index_col=0)
    pval.append(df)

In [ ]:
pval_frame = pd.concat(pval, axis=0, ignore_index=True)
pval_frame

In [ ]:
pval_frame.to_csv(file_path+'Healthy_PAC_chord_figure_input_before_filter_pval_lastestDataBase.csv') 